In [3]:
import pandas as pd

# Load the Excel file
with pd.ExcelFile('variable coding.xlsx') as xls:
    # Get sheet names
    sheet_names = xls.sheet_names

    # Loop over each sheet
    for sheet in sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)

        # Add 'Abbrev' column
        df['Abbrev'] = df['Detail'].apply(lambda x: '_'.join([x.split('!!')[0].lower(),
                                                              '_'.join(x.split('!!')[-1].lower().split())]))

        # Save each sheet back to the Excel file
        with pd.ExcelWriter('variable coding.xlsx', engine='openpyxl', mode='a') as writer: 
            df.to_excel(writer, sheet_name=sheet, index=False)

print('Excel file successfully updated.')

ValueError: Sheet '2021' already exists and if_sheet_exists is set to 'error'.

In [7]:
## read 'variable coding.xlsx'

df = pd.read_excel('variable coding.xlsx', sheet_name='2021')

## see the data type of each column

df['Detail']





0                     Estimate !RACE ! !Total population
1                     Percent! !RACE ! !Total population
2       Estimate ! !RACE ! !Total population! ! One race
3          Percent! !RACE! !Total population! ! One race
4      Estimate ! !RACE ! !Total population! ! One ra...
                             ...                        
149    Percent! INCOME AND BENEFITS (IN 2021 INFLATIO...
150    Estimate! INCOME AND BENEFITS (IN 2021 INFLATI...
151    Percent! INCOME AND BENEFITS (IN 2021 INFLATIO...
152    Estimate! INCOME AND BENEFITS (IN 2021 INFLATI...
153    Percent! !PERCENTAGE OF FAMILIES AND PEOPLE WH...
Name: Detail, Length: 154, dtype: object

In [10]:
import pandas as pd

# Load the Excel file
with pd.ExcelFile('variable coding (1).xlsx') as xls:
    # Get sheet names
    sheet_names = xls.sheet_names

    # Create a new Excel writer object
    with pd.ExcelWriter('census_metadata_updated.xlsx', engine='openpyxl') as writer:
        # Loop over each sheet
        for sheet in sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet)

            # Add 'Abbrev' column
            df['Abbrev'] = df['Detail'].apply(lambda x: '_'.join([x.split('!!')[0].lower(),
                                                                  '_'.join(x.split('!!')[-1].lower().split())]))

            # Save each sheet to the new Excel file
            df.to_excel(writer, sheet_name=sheet, index=False)

print('New Excel file successfully created.')

New Excel file successfully created.


In [14]:
import pandas as pd
import re

# Function to clean abbreviations
def clean_abbrev(abbrev):
    words = abbrev.split('_')

    # Replace 'percent' or 'number' with 'pct' or 'est'
    if words[0] == 'percent':
        words[0] = 'pct'
    else:
        words[0] = 'est'
    
    # Remove non-letter characters except underscore
    words = [re.sub(r'[^\w\s]', '', word) for word in words]

    # Keep at most 3 words after the first word
    return '_'.join(words[:4])

# Load the Excel file
with pd.ExcelFile('variable coding (2).xlsx') as xls:
    # Get sheet names
    sheet_names = xls.sheet_names

    # Create a new Excel writer object
    with pd.ExcelWriter('census_metadata_updated.xlsx', engine='openpyxl') as writer:
        # Loop over each sheet
        for sheet in sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet)

            # Add 'Abbrev' column
            df['Abbrev'] = df['Detail'].apply(lambda x: '_'.join([x.split('!!')[0].lower(),
                                                                  '_'.join(x.split('!!')[-1].lower().split())]))

            # Clean 'Abbrev' column
            df['Abbrev'] = df['Abbrev'].apply(clean_abbrev)

            # Save each sheet to the new Excel file
            df.to_excel(writer, sheet_name=sheet, index=False)

print('New Excel file successfully created.')

New Excel file successfully created.


In [19]:
import pandas as pd
import re

# Function to clean abbreviations
def clean_abbrev(abbrev):
    words = abbrev.split('_')

    # Replace 'percent' with 'pct' and other with 'est'

    ## if the first word *contains* 'percent'
    if words[0].find('percent') != -1:
        words[0] = 'pct'
    else:
        words[0] = 'est'

    # Remove non-letter characters except underscore
    words = [re.sub(r'[^\w\s]', '', word) for word in words]

    # Keep at most 3 words after the first word
    words = words[:4]

    # remove last word when it is hanging
    if words[-1] in ["and", "or", "no", "except", "excluding"]:
        words = words[:-1]

    return '_'.join(words)

# Load the Excel file
with pd.ExcelFile('variable coding.xlsx') as xls:
    # Get sheet names
    sheet_names = xls.sheet_names

    # Create a new Excel writer object
    with pd.ExcelWriter('census_metadata_updated.xlsx', engine='openpyxl') as writer:
        # Loop over each sheet
        for sheet in sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet)

            # Add 'Abbrev' column
            df['Abbrev'] = df['Detail'].apply(lambda x: '_'.join([x.split('!!')[0].lower(),
                                                                  '_'.join(x.split('!!')[-1].lower().split())]))

            # Clean 'Abbrev' column
            df['Abbrev'] = df['Abbrev'].apply(clean_abbrev)

            # Check for duplicates in 'Abbrev'
            if df['Abbrev'].duplicated().any():
                print(f'Duplicates found in sheet {sheet}')
                # Append the last word from 'Detail' to the 'Abbrev' for duplicates
                dup_rows = df[df['Abbrev'].duplicated(keep=False)]
                for idx, row in dup_rows.iterrows():
                    df.loc[idx, 'Abbrev'] += "_" + row['Detail'].split('!!')[-1].split()[-1].lower()


            # Save each sheet to the new Excel file
            df.to_excel(writer, sheet_name=sheet, index=False)

print('New Excel file successfully created.')


Duplicates found in sheet 2021
Duplicates found in sheet 2020
Duplicates found in sheet 2019
Duplicates found in sheet 2018
Duplicates found in sheet 2017
Duplicates found in sheet 2016
Duplicates found in sheet 2015
Duplicates found in sheet 2014
Duplicates found in sheet 2013
Duplicates found in sheet 2012
Duplicates found in sheet 2011
Duplicates found in sheet 2010
Duplicates found in sheet 2009
New Excel file successfully created.


In [20]:
import pandas as pd
import json

# Load the Excel file
with pd.ExcelFile('census_metadata_updated.xlsx') as xls:
    # Get sheet names
    sheet_names = xls.sheet_names

    # Read each sheet into a separate DataFrame
    all_data = {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_names}

# Create a set for all shared Abbrevs
shared_abbrevs = set(all_data[sheet_names[0]]['Abbrev'])

# Find shared Abbrevs across all years
for sheet in sheet_names[1:]:
    shared_abbrevs &= set(all_data[sheet]['Abbrev'])

# Create a dictionary to store Category and Abbrev relationships
category_abbrev_dict = {}

# Only consider shared Abbrevs
for sheet in sheet_names:
    for idx, row in all_data[sheet].iterrows():
        if row['Abbrev'] in shared_abbrevs:
            if row['Category'] in category_abbrev_dict:
                if row['Abbrev'] not in category_abbrev_dict[row['Category']]:
                    category_abbrev_dict[row['Category']].append(row['Abbrev'])
            else:
                category_abbrev_dict[row['Category']] = [row['Abbrev']]

# Write to a JSON file
with open('category_abbrev.json', 'w') as f:
    json.dump(category_abbrev_dict, f, indent=4)

# Create a dictionary to store year and Code-Abbrev relationships
year_code_abbrev_dict = {}

for sheet in sheet_names:
    year_code_abbrev_dict[sheet] = {row['Code']: row['Abbrev'] for idx, row in all_data[sheet].iterrows()}

# Write to a JSON file
with open('year_code_abbrev.json', 'w') as f:
    json.dump(year_code_abbrev_dict, f, indent=4)
